## Visualise stokes annulus model (flow etc)

In [1]:
# to fix trame issue
import nest_asyncio
nest_asyncio.apply()
import os

In [2]:
import petsc4py
import underworld3 as uw
import numpy as np
import sympy

In [3]:
ls -trl output/annulus/ | tail

total 0
drwxr-xr-x  2385 lmoresi  staff  76320 Aug 13 21:37 Ra1e5/


In [4]:
ls -trl output/annulus/Ra1e5/ | tail

-rw-r--r--    1 lmoresi  staff  220952 Aug 13 21:37 annulus_ckpt.mesh.T.00592.h5
-rw-r--r--    1 lmoresi  staff    3030 Aug 13 21:37 annulus_ckpt.mesh.00592.xdmf
-rw-r--r--    1 lmoresi  staff  129560 Aug 13 21:37 annulus_ckpt.mesh.U.00593.h5
-rw-r--r--    1 lmoresi  staff   49576 Aug 13 21:37 annulus_ckpt.mesh.P.00593.h5
-rw-r--r--    1 lmoresi  staff  220952 Aug 13 21:37 annulus_ckpt.mesh.T.00593.h5
-rw-r--r--    1 lmoresi  staff    3030 Aug 13 21:37 annulus_ckpt.mesh.00593.xdmf
-rw-r--r--    1 lmoresi  staff  129560 Aug 13 21:37 annulus_ckpt.mesh.U.00594.h5
-rw-r--r--    1 lmoresi  staff   49576 Aug 13 21:37 annulus_ckpt.mesh.P.00594.h5
-rw-r--r--    1 lmoresi  staff  220952 Aug 13 21:37 annulus_ckpt.mesh.T.00594.h5
-rw-r--r--    1 lmoresi  staff    3030 Aug 13 21:37 annulus_ckpt.mesh.00594.xdmf


In [5]:
checkpoint_dir = "output/annulus/Ra1e5"
checkpoint_base = f"annulus_ckpt"
meshfile = os.path.join(checkpoint_dir, checkpoint_base) + ".mesh.00000.h5"
image_dir = os.path.join("output/annulus/Ra1e5","images")
os.makedirs(image_dir, exist_ok=True)

In [6]:
r_i = 0.5
r_o = 1.0

mesh = uw.discretisation.Mesh(meshfile, 
                              coordinate_system_type=uw.coordinates.CoordinateSystemType.CYLINDRICAL2D)

x = mesh.N.x
y = mesh.N.y

r = sympy.sqrt(x**2 + y**2)  # cf radius_fn which is 0->1
th = sympy.atan2(y + 1.0e-5, x + 1.0e-5)

v_soln = uw.discretisation.MeshVariable("U", mesh, mesh.dim, degree=2)
t_soln = uw.discretisation.MeshVariable("T", mesh, 1, degree=3)
flux = uw.discretisation.MeshVariable(r"dTdz", mesh, 1, degree=1)

In [12]:
steps = range(600,800,1)

In [13]:
import mpi4py
import pyvista as pv
import underworld3.visualisation as vis

pl = pv.Plotter(window_size=[750, 750])

for step in steps:

    try:
        v_soln.read_timestep(checkpoint_base, "U", step, outputPath=checkpoint_dir)
        t_soln.read_timestep(checkpoint_base, "T", step, outputPath=checkpoint_dir)
        print(f"Plotting step {step}")

    except:
        print(f"Failed to read step {step} correctly")
        continue

    pl.clear()

    pvmesh = vis.mesh_to_pv_mesh(mesh)
    # pvmesh.point_data["T"] = vis.scalar_fn_to_pv_points(pvmesh, t_soln.sym[0])
    pvmesh.point_data["V"] = vis.vector_fn_to_pv_points(pvmesh, v_soln.sym)

    pvmesh_v = vis.meshVariable_to_pv_mesh_object(v_soln, alpha=None)
    pvmesh_t = vis.meshVariable_to_pv_mesh_object(t_soln, alpha=None)

    pvmesh_v.point_data["V"] = vis.vector_fn_to_pv_points(pvmesh_v, v_soln.sym)
    pvmesh_t.point_data["T"] = vis.scalar_fn_to_pv_points(pvmesh_t, t_soln.sym)
    
    # point sources at cell centres
    skip = 3
    points = np.zeros((mesh._centroids[::skip].shape[0], 3))
    points[:, 0] = mesh._centroids[::skip, 0]
    points[:, 1] = mesh._centroids[::skip, 1]
    point_cloud = pv.PolyData(points)
    
    pvstream = pvmesh.streamlines_from_source(
        point_cloud,
        vectors="V",
        integration_direction="both",
        max_time=1,
        surface_streamlines=True,
    )
    
    pl.add_mesh(
        pvmesh_t,
        cmap="RdBu_r",
        edge_color="Black",
        edge_opacity=0.1,
        show_edges=True,
        scalars="T",
        use_transparency=False,
        opacity=1.0,
        show_scalar_bar=False
    )
    
    pl.add_mesh(pvstream, opacity=0.4, show_scalar_bar=False)    

    imagefile = os.path.join(image_dir, checkpoint_base) + f"_V_T_{step}.png"
    
    pl.screenshot(filename=imagefile, window_size=(1000, 1000), return_img=False)



Plotting step 600
Plotting step 601
Plotting step 602
Plotting step 603
Plotting step 604
Plotting step 605
Plotting step 606
Plotting step 607
Plotting step 608
Plotting step 609
Plotting step 610
Plotting step 611
Plotting step 612
Plotting step 613
Plotting step 614
Plotting step 615
Plotting step 616
Plotting step 617
Plotting step 618
Plotting step 619
Plotting step 620
Plotting step 621
Plotting step 622
Plotting step 623
Plotting step 624
Plotting step 625
Plotting step 626
Plotting step 627
Plotting step 628
Plotting step 629
Plotting step 630
Plotting step 631
Plotting step 632
Plotting step 633
Plotting step 634
Plotting step 635
Plotting step 636
Plotting step 637
Plotting step 638
Plotting step 639
Plotting step 640
Plotting step 641
Plotting step 642
Plotting step 643
Plotting step 644
Plotting step 645
Plotting step 646
Plotting step 647
Plotting step 648
Plotting step 649
Plotting step 650
Plotting step 651
Plotting step 652
Plotting step 653
Plotting step 654
Plotting s

In [9]:
pl.show()

Widget(value='<iframe id="pyvista-jupyter_trame__template_P_0x16d666b10_0" src="http://localhost:8889/trame-ju…

In [10]:
## Calculate heat flux, evaluate at surface — proxy for boundary layer thickness

ZeroDivisionError: division by zero

In [ ]:
flux_solver = uw.systems.Projection(mesh, flux)

# Conductive flux only !
radial_flux = -mesh.vector.gradient(t_soln.sym[0]).dot(mesh.CoordinateSystem.unit_e_0)
radial_flux *= sympy.exp(-100*(r-1)**2) 

flux_solver.uw_function = radial_flux
flux_solver.smoothing = 1.0e-3
flux_solver.solve()

In [ ]:
import mpi4py
import pyvista as pv
import underworld3.visualisation as vis

pvmesh = vis.mesh_to_pv_mesh(mesh)
pvmesh.point_data["dTdz"] = vis.scalar_fn_to_pv_points(pvmesh, flux.sym[0])
pvmesh.point_data["V"] = vis.vector_fn_to_pv_points(pvmesh, v_soln.sym)
pvmesh.point_data["V"] -= pvmesh.point_data["V"].mean()

velocity_points = vis.meshVariable_to_pv_cloud(v_soln)
velocity_points.point_data["V"] = vis.vector_fn_to_pv_points(velocity_points, v_soln.sym)

# point sources at cell centres
skip = 8
points = np.zeros((mesh._centroids[::skip].shape[0], 3))
points[:, 0] = mesh._centroids[::skip, 0]
points[:, 1] = mesh._centroids[::skip, 1]
point_cloud = pv.PolyData(points)

pvstream = pvmesh.streamlines_from_source(
    point_cloud,
    vectors="V",
    integration_direction="both",
    max_time=0.5,
)

pl = pv.Plotter(window_size=[1000, 1000])

pl.add_mesh(
    pvmesh,
    cmap="Reds",
    edge_color="Black",
    show_edges=True,
    scalars="dTdz",
    use_transparency=False,
    opacity=1.0,
)

# 
# pl.add_mesh(pvmesh, "Black", "wireframe",  opacity=0.1)

# pl.add_points(point_cloud, color="White", point_size=3.0, opacity=0.25)

# pl.add_arrows(velocity_points.points, velocity_points.point_data["V"], mag=0.0001, show_scalar_bar=True)
pl.add_mesh(pvstream, opacity=0.4, show_scalar_bar=False)
    
# pl.remove_scalar_bar("V")

imagefile = os.path.join(image_dir, checkpoint_base) + f"_dTdr_{step}.png"

pl.screenshot(filename=imagefile, window_size=(1000, 1000), return_img=False)
# OR


In [ ]:
pl.show()

In [ ]:
! open .